In [4]:
# Install required packages
!pip install scipy tensorflow seaborn

import numpy as np
import scipy.signal as signal
import pickle
import os
from sklearn.preprocessing import normalize, StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns



In [8]:
# Set data path
data_path = r'D:\SEM_6\DL_SIP\pjt\data_preprocessed_python\data_preprocessed_python'

# Function to compute band power using scipy
def bandpower(data, sf, band, window_sec=4, relative=False):
    band = np.asarray(band)
    low, high = band

    # Compute the PSD
    freqs, psd = signal.welch(data, sf, nperseg=window_sec*sf)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using Simpson's rule.
    bp = np.trapz(psd[idx_band], dx=freq_res)

    if relative:
        bp /= np.trapz(psd, dx=freq_res)
    return bp

In [9]:
# FFT Processing function
def FFT_Processing(sub, channel, band, window_size, step_size, sample_rate):
    meta = []
    with open(os.path.join(data_path, 's' + sub + '.dat'), 'rb') as file:
        subject = pickle.load(file, encoding='latin1')
        for i in range(0, 40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0
            while start + window_size < data.shape[1]:
                meta_data = []
                for j in channel:
                    X = data[j][start : start + window_size]
                    Y = [bandpower(X, sample_rate, [band[k], band[k+1]]) for k in range(len(band) - 1)]
                    meta_data = meta_data + list(Y)
                meta.append([meta_data, labels])
                start = start + step_size
        meta = np.array(meta, dtype=object)
        np.save(os.path.join(data_path, 's' + sub), meta, allow_pickle=True, fix_imports=True)


In [10]:
# Define parameters
channel = [1, 2, 3, 4, 6, 11, 13, 17, 19, 20, 21, 25, 29, 31]
band = [4, 8, 12, 16, 25, 45]
window_size = 256
step_size = 16
sample_rate = 128
subjectList = ['01', '02', '03']

# Process data for each subject
for subjects in subjectList:
    FFT_Processing(subjects, channel, band, window_size, step_size, sample_rate)


In [11]:
# Load and prepare data
data_training = []
label_training = []
data_testing = []
label_testing = []

for subjects in subjectList:
    with open(os.path.join(data_path, 's' + subjects + '.npy'), 'rb') as file:
        sub = np.load(file, allow_pickle=True)
        for i in range(sub.shape[0]):
            if i % 4 == 0:
                data_testing.append(sub[i][0])
                label_testing.append(sub[i][1])
            else:
                data_training.append(sub[i][0])
                label_training.append(sub[i][1])

# Save the split data
np.save(os.path.join(data_path, 'data_training'), np.array(data_training), allow_pickle=True, fix_imports=True)
np.save(os.path.join(data_path, 'label_training'), np.array(label_training), allow_pickle=True, fix_imports=True)
np.save(os.path.join(data_path, 'data_testing'), np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save(os.path.join(data_path, 'label_testing'), np.array(label_testing), allow_pickle=True, fix_imports=True)



In [12]:
# Load training data
with open(os.path.join(data_path, 'data_training.npy'), 'rb') as fileTrain:
    X = np.load(fileTrain)
with open(os.path.join(data_path, 'label_training.npy'), 'rb') as fileTrainL:
    Y = np.load(fileTrainL)



In [13]:
# Normalize and preprocess the data
X = normalize(X)
Z = np.ravel(Y[:, [3]])
y_train = to_categorical(Z)
x_train = np.array(X[:])



In [14]:
# Load testing data
with open(os.path.join(data_path, 'data_testing.npy'), 'rb') as fileTrain:
    M = np.load(fileTrain)
with open(os.path.join(data_path, 'label_testing.npy'), 'rb') as fileTrainL:
    N = np.load(fileTrainL)

M = normalize(M)
L = np.ravel(N[:, [3]])
y_test = to_categorical(L)
x_test = np.array(M[:])



In [15]:
# Standardize the data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

# Reshape data to fit the GRU model
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)



In [18]:
# Build the GRU model
model = Sequential()
model.add(GRU(164, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.6))
model.add(GRU(256, return_sequences=True))
model.add(Dropout(0.6))
model.add(GRU(82, return_sequences=True))
model.add(Dropout(0.6))
model.add(GRU(42))
model.add(Dropout(0.4))
model.add(Dense(21, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 70, 164)           82164     
                                                                 
 dropout_4 (Dropout)         (None, 70, 164)           0         
                                                                 
 gru_5 (GRU)                 (None, 70, 256)           324096    
                                                                 
 dropout_5 (Dropout)         (None, 70, 256)           0         
                                                                 
 gru_6 (GRU)                 (None, 70, 82)            83640     
                                                                 
 dropout_6 (Dropout)         (None, 70, 82)            0         
                                                                 
 gru_7 (GRU)                 (None, 42)               

In [ ]:
# Train the model
m = model.fit(x_train, y_train, epochs=50, batch_size=256, verbose=1, validation_data=(x_test, y_test))


Epoch 1/50
 58/172 [=========>....................] - ETA: 1:59 - loss: 2.0513 - accuracy: 0.2280

In [ ]:
# Evaluate the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(m.history['accuracy'], label='Train Accuracy', color='green')
plt.plot(m.history['val_accuracy'], label='Test Accuracy', color='red')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(m.history['loss'], label='Train Loss', color='green')
plt.plot(m.history['val_loss'], label='Test Loss', color='red')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(data_path, "GRU_Model_Performance.png"))
plt.show()



In [ ]:
# Save the model
model.save(os.path.join(data_path, 'gru_model.h5'))
np.save(os.path.join(data_path, 'gru_model_history.npy'), m.history)



In [ ]:
# Confusion Matrix
y_pred = model.predict(x_test)
y_test1 = np.argmax(y_test, axis=1)
y_pred1 = np.argmax(y_pred, axis=1)
cmatrix = confusion_matrix(y_test1, y_pred1)

plt.figure(figsize=(10, 8))
sns.heatmap(cmatrix, annot=True, fmt='d', cmap='Blues')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.savefig(os.path.join(data_path, "GRU_Model_Confusion_Matrix.png"))
plt.show()



In [ ]:
# Classification Report
print(classification_report(y_test1, y_pred1))
